<a href="https://colab.research.google.com/github/ismoil27/jaydariGPT/blob/main/jaydari_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torch bitsandbytes datasets

In [2]:
from transformers import AutoTokenizer, BitsAndBytesConfig, AutoModelForCausalLM
import torch
from datasets import load_dataset

In [ ]:
model_id = 'TinyLlama/TinyLlama-1.1B-Chat-v1.0'
tokenizer = AutoTokenizer.from_pretrained(model_id)

# print('Vocab size:', tokenizer.vocab_size)
# print('Special tokens:', tokenizer.special_tokens_map)

# quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type='nf4'
)

bnb_config

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map='auto', # GPU, CPU
    # dtype=torch.bfloat16
)

In [ ]:
# Before Fine-tuning
prompt = "Explain what a tokenizer is?"
# prompt = "A tokenizer is a tool in natural language processing that"

inputs = tokenizer(prompt, return_tensors="pt").to(model.device) # GPU, CPU

with torch.no_grad():
    output_ids = model.generate(
        **inputs,
        max_new_tokens=80,
        do_sample=True,
        temperature=0.7
    )

print(tokenizer.decode(output_ids[0], skip_special_tokens=True))

# print(model)
first_block = model.model.layers[0]
print('first_block:', first_block)
print('=======')
print(first_block.self_attn)
print('=======')
print(model.config)



In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

total_params = count_parameters(model)
print(f"Total parameters (including frozen 4-bit): {total_params:,}")


## datasets library | load_dataset
* instruction tuning

In [ ]:
from datasets import load_dataset

dataset = load_dataset("yahma/alpaca-cleaned", split="train")
dataset
dataset[1]

In [ ]:
def generate_prompt(example):
  instruction = example['instruction']
  input_text = example['input']
  output_text = example['output']

  if input_text:
    return(
        "### Instruction:\n"
        f"{instruction}\n\n"
        "### Input:\n"
        f"{input_text}\n\n"
        "### Response:\n"
        f"{output_text}"
    )
  else:
    return(
       "### Instruction:\n"
       f"{instruction}\n\n"
       "### Response:\n"
       f"{output_text}"
    )

# generate_prompt(dataset[0])

def formatting_func(example):
  return {'text': generate_prompt(example)}

dataset = dataset.map(formatting_func)


In [ ]:
dataset[0]['text']

In [9]:
dataset = dataset.select(range(7000))

In [10]:
dataset = dataset.shuffle(seed=42)

In [11]:
# [7, 3, 2, 8, 5, 6, 9, 4, 0, 1]
# [7, 3, 2, 8, 5, 6, 9, 4, 0, 1] EXACT SAME ORDER

In [12]:
# [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
# [10, 231, 342, 3453, 3464, 5123, 6456, 7, 8, 9]


In [13]:
dataset

Dataset({
    features: ['output', 'input', 'instruction', 'text'],
    num_rows: 7000
})